In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
import os
import numpy as np
import pandas as pd
from scipy.spatial.distance import cdist

In [ ]:
BASE_DIR = "/content/drive/MyDrive/Colab Notebooks/DLS-1/Project"

CELEBA_DIR    = os.path.join(BASE_DIR, "CelebA")

IDENTITY_TXT  = os.path.join(CELEBA_DIR, "identity_CelebA.txt")
LANDMARKS_TXT = os.path.join(CELEBA_DIR, "list_landmarks_celeba.txt")
BBOX_CSV      = os.path.join(CELEBA_DIR, "list_bbox_celeba.csv")
LIST_ATTR_CSV = os.path.join(CELEBA_DIR, "list_attr_celeba.csv")

SELECTED_CSV  = os.path.join(BASE_DIR, "selected_celeba.csv")
TRAIN_CSV     = os.path.join(BASE_DIR, "train_celeba.csv")
VAL_CSV       = os.path.join(BASE_DIR, "val_celeba.csv")

## Шаг 1
* Зачитываем файлы identity_CelebA.txt, list_landmarks_celeba.txt, list_bbox_celeba.csv, list_attr_celeba.csv.
* Контролируем отсутствие дублей значений в колонке "image_id".
* Соединяем внутренним соединением по "image_id" в одну таблицу для дальнейшего анализа.

In [ ]:
# читаем четыре таблицы и соединяем внутренним соединением по "image_id"
id_df = pd.read_csv(
    IDENTITY_TXT,
    sep=r"\s+",
    header=None,
    names=["image_id", "person_id"]
)
assert(id_df["image_id"].is_unique)

lm_column_names = [
    "image_id",
    "lefteye_x", "lefteye_y",
    "righteye_x", "righteye_y",
    "nose_x", "nose_y",
    "leftmouth_x", "leftmouth_y",
    "rightmouth_x", "rightmouth_y"
]
lm_df = pd.read_csv(LANDMARKS_TXT, sep=r"\s+", skiprows=2, header=0, names=lm_column_names)
assert(lm_df["image_id"].is_unique)

bbox_df = pd.read_csv(BBOX_CSV)
assert(bbox_df["image_id"].is_unique)

attr_df = pd.read_csv(LIST_ATTR_CSV)
assert(attr_df["image_id"].is_unique)

# inner join по image_id
data_df = (id_df
           .merge(lm_df,   on="image_id", how="inner")
           .merge(bbox_df, on="image_id", how="inner")
           .merge(attr_df, on="image_id", how="inner"))

print(f"Уникальных персон: {data_df["person_id"].nunique()}")
print(f"Всего изображений: {len(data_df)}")

Уникальных персон: 10177
Всего изображений: 202598


## Шаг 2
* Проверяем относительные расстояния (по X) между глазами и носом. Цель - отобрать только фото с лицами, обращенными прямо на нас, или почти так.
* Проверяем абсолютное расстояние (по X) между глазами. Цель - исключить очень мелкие лица, у которых почти нет деталей.
* Среди изображений, прошедших фильтрацию, отбираем фото только тех персон, у которых осталось не менее 20 фото.

In [ ]:
# lefteye_delta_x: расстояние от левого глаза до носа по горизонтали
data_df["lefteye_delta_x"] = data_df["nose_x"] - data_df["lefteye_x"]

# righteye_delta_x: расстояние от носа до правого глаза по горизонтали
data_df["righteye_delta_x"] = data_df["righteye_x"] - data_df["nose_x"]

# eyes_delta_x: общее расстояние между глазами
data_df["eyes_delta_x"] = data_df["righteye_x"] - data_df["lefteye_x"]

# Условие 1: Все дельты больше нуля (гарантирует правильный порядок точек: левый глаз -> нос -> правый глаз)
min_eyes_delta_x = 30
cond1 = (data_df["lefteye_delta_x"] > 0) & \
        (data_df["righteye_delta_x"] > 0) & \
        (data_df["eyes_delta_x"] > min_eyes_delta_x)

# Условие 2: Нос не слишком близко к краям (не менее 30% расстояния между глазами с каждой стороны)
threshold = 0.30
cond2 = (data_df["lefteye_delta_x"] >= threshold * data_df["eyes_delta_x"]) & \
        (data_df["righteye_delta_x"] >= threshold * data_df["eyes_delta_x"])

data_df = data_df[cond1 & cond2]

data_df = data_df.drop(columns=["lefteye_delta_x", "righteye_delta_x", "eyes_delta_x"])

print(f"Уникальных персон: {data_df["person_id"].nunique()}")
print(f"Всего изображений: {len(data_df)}")
#print(f"Процент отсева: {100 * (1 - len(filtered_df)/len(data_df)):.2f}%")

Уникальных персон: 10089
Всего изображений: 146644


In [ ]:
# считаем количество фото на персону
person_counts = (
    data_df
    .groupby("person_id")
    .size()
    .reset_index(name="num_images")
)

# оставляем персон с >= 20 изображениями
valid_persons = person_counts[person_counts["num_images"] >= 20]["person_id"]

data_df = data_df[data_df["person_id"].isin(valid_persons)]

print(f"Уникальных персон: {data_df["person_id"].nunique()}")
print(f"Всего изображений: {len(data_df)}")

Уникальных персон: 2910
Всего изображений: 67750


## Шаг 3

Из отобранных на первых двух шагах персон выбираем случайных 500 но т.о., чтобы примерно сохранилась частота, с которой встречаются значения четырёх самых "смысловых" атрибутов (мы их сами просто выбрали, интуитивно + по подсказке ИИ, эти 4 атрибута: пол, молодой, улыбается, в очках). Т.е. пытаемся сделать выборку репрезентативной в рамках этих 4-х атрибутов.

А именно, последовательно:
*   агрегируем атрибуты по персоне (считаем среднее для каждого из 40 бинарных атрибутов)
*   используем четыре назначенных «смысловыми» атрибутов для стратификации, бинаризируем их значения (>=0 → 1, <0 → 0)
*   получаем 2**4 = 16 групп персон ("страт", от 0000 до 1111), для каждой вычисляем её долю в общем наборе персон
*   для каждой из 16 "страт" случайным образом выбираем количество персон, пропорциональное их доле в исходном наборе (всего выбираем 500 персон суммарно по всем "стратам")

In [ ]:
# список атрибутов (все, кроме image_id и person_id)
attr_cols = [c for c in attr_df.columns if c not in ["image_id"]]

# агрегируем атрибуты по персоне (доли +1)
person_attr = (
    data_df
    .groupby("person_id")[attr_cols]
    .mean()
    .reset_index()
)

# ключевые атрибуты для стратификации
key_attrs = ["Male", "Young", "Smiling", "Eyeglasses"]

# бинaризация: >=0 → 1, <0 → 0
for col in key_attrs:
    person_attr[col] = (person_attr[col] >= 0).astype(int)

# создаём стратификационную группу
person_attr["strata"] = person_attr[key_attrs].astype(str).agg("_".join, axis=1)

# сколько персон брать из каждой страты
n_target = 500
strata_counts = person_attr["strata"].value_counts(normalize=True)
strata_quota = (strata_counts * n_target).round().astype(int)

# отбор персон
selected_persons = []

rng = np.random.default_rng(42)

for strata, k in strata_quota.items():
    candidates = person_attr[person_attr["strata"] == strata]["person_id"]
    if len(candidates) == 0:
        continue
    selected_persons.extend(
        rng.choice(
            candidates,
            size=min(k, len(candidates)),
            replace=False
        )
    )

# если набрали больше / меньше 500 — аккуратно подрезаем / добираем
selected_persons = list(dict.fromkeys(selected_persons))  # уникальные

if len(selected_persons) > 500:
    selected_persons = selected_persons[:500]
elif len(selected_persons) < 500:
    remaining = list(
        set(person_attr["person_id"]) - set(selected_persons)
    )
    selected_persons.extend(
        rng.choice(
            remaining,
            size=500 - len(selected_persons),
            replace=False
        )
    )

print(f"Выбрано персон: {len(selected_persons)}")


Выбрано персон: 500


## Шаг 4

Для каждой из 500 персон, выбранных на предыдущем шаге, выбираем 20 изображений с максимальным разнообразием атрибутов (это очень условно, т.к. выбираем почти все фото, отсеиваем малую часть).

А именно, для каждой персоны:
*   случайно выбираем первое изображение
*   в цикле выбираем по одному дополнительному изображению, пока количество изображений не достигнет 20
*   критерий выбора очередного изображения: максимизация "минимального расстояния" от каждого из уже отобранных изображений до выбираемого следующим изображения (жадный алгоритм max–min по расстоянию Хэмминга)

In [ ]:
def select_maxmin_samples(attr_matrix, k=20):
    """
    attr_matrix: (N, D) numpy array with values in {-1, 1}
    returns indices of selected samples
    """
    n = attr_matrix.shape[0]
    selected = []

    # 1) случайно выбираем первый элемент
    first = np.random.randint(n)
    selected.append(first)

    # 2) greedy max-min
    for _ in range(1, k):
        remaining = list(set(range(n)) - set(selected))
        dists = cdist(
            attr_matrix[remaining],
            attr_matrix[selected],
            metric="hamming"
        )
        min_dists = dists.min(axis=1)
        next_idx = remaining[np.argmax(min_dists)]
        selected.append(next_idx)

    return selected

In [ ]:
# основной цикл по персонам
selected_rows = []

for person_id in selected_persons:
    person_df = data_df[data_df["person_id"] == person_id]

    attrs = person_df[attr_cols].values  # (N, 40)
    selected_idx = select_maxmin_samples(attrs, k=20)

    selected_rows.append(person_df.iloc[selected_idx])

# финальный датасет
final_df = pd.concat(selected_rows, ignore_index=True)

# считаем количество изображений на персону
counts = final_df["person_id"].value_counts()

print(counts.describe())

# проверка: все ли ровно по 20
assert (counts == 20).all(), "Ошибка: не у всех персон ровно 20 изображений"

print("✅ Проверка пройдена: у каждой персоны ровно 20 изображений")

print(f"Уникальных персон: {final_df["person_id"].nunique()}")
print(f"Всего изображений: {len(final_df)}")

count    500.0
mean      20.0
std        0.0
min       20.0
25%       20.0
50%       20.0
75%       20.0
max       20.0
Name: count, dtype: float64
✅ Проверка пройдена: у каждой персоны ровно 20 изображений
Уникальных персон: 500
Всего изображений: 10000


## Шаг 5

Вычисляем средние значения атрибутов по отобранным 10 000 изображений, сравниваем с средними значениями по исходному полному набору изображений.

Средние значения атрибутов в результате выбора 10 000 изображений изменились, но не очень значительно, принимаем.

In [ ]:
# проверяем баланс атрибутов, сравниваем c балансом полного набора
attr_means = pd.DataFrame({
    "All 200K+ images": attr_df[attr_cols].mean(),
    "Selected 10K images": final_df[attr_cols].mean(),
    "Delta": final_df[attr_cols].mean() - attr_df[attr_cols].mean()
}).round(2)

attr_means = attr_means.sort_values(by="Selected 10K images", ascending=True)

print(attr_means)

                     All 200K+ images  Selected 10K images  Delta
Blurry                          -0.90                -0.98  -0.08
Bald                            -0.96                -0.97  -0.02
Gray_Hair                       -0.92                -0.93  -0.02
Wearing_Hat                     -0.90                -0.93  -0.03
Mustache                        -0.92                -0.91   0.01
Chubby                          -0.88                -0.90  -0.01
Eyeglasses                      -0.87                -0.90  -0.03
Double_Chin                     -0.91                -0.90   0.01
Pale_Skin                       -0.91                -0.89   0.03
Goatee                          -0.87                -0.88  -0.00
Sideburns                       -0.89                -0.88   0.01
Receding_Hairline               -0.84                -0.86  -0.02
Wearing_Necktie                 -0.85                -0.86  -0.01
Rosy_Cheeks                     -0.87                -0.80   0.07
Narrow_Eye

## Шаг 6
* Добавляем колонку "label" с уникальным номером 0..499 для каждой персоны, для будущего использования.
* Разбиваем все отобранные 1000 фото на train и val части в пропорции 80/20.
* Сохраняем таблицы в файлы: selected_celeba.csv, train_celeba.csv, val_celeba.csv.

In [ ]:
unique_person_ids = sorted(final_df["person_id"].unique())
person_id_2_label = {pid: i for i, pid in enumerate(unique_person_ids)}

final_df["label"] = final_df["person_id"].map(person_id_2_label)

In [ ]:
train_parts = []
val_parts = []

for _, group in final_df.groupby("person_id"):
    # делаем shuffle 20 изображений внутри персоны
    group = group.sample(frac=1, random_state=42)
    n_train = int(0.8 * len(group)) # в train берем случайные 16 из 20
    train_parts.append(group.iloc[:n_train])
    val_parts.append(group.iloc[n_train:])

train_df = pd.concat(train_parts, ignore_index=True)
val_df = pd.concat(val_parts, ignore_index=True)

# делаем shuffle выборки train
train_df = train_df.sample(frac=1, random_state=42).reset_index(drop=True)

print("Train:", len(train_df))
print("Val:", len(val_df))

Train: 8000
Val: 2000


In [ ]:
final_df.to_csv(SELECTED_CSV, index=False)
train_df.to_csv(TRAIN_CSV, index=False)
val_df.to_csv(VAL_CSV, index=False)

print(f"Файлы сохранёны")

Файлы сохранёны
